# Test: Conditional Colab Install Pattern

Verifies that the install cell correctly:
- **On Colab**: skips core packages (uses pre-installed ones), only installs non-Colab packages
- **Locally**: installs core packages at Colab's exact versions, creating a matching environment

After running: check that all versions match the expected Colab versions and C extensions work.

In [ ]:
import subprocess, sys
def _pip(*a): subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', *a])

# Packages NOT pre-installed on Colab (always install everywhere)
_pip('colorama==0.4.6')

# Core packages (pre-installed on Colab, install locally to match Colab env)
if 'google.colab' not in sys.modules:
    _pip('numpy==2.0.2', 'pandas==2.2.2', 'scikit-learn==1.6.1', 'scipy==1.16.3', 'matplotlib==3.10.0')

In [ ]:
import sys
import numpy as np
import pandas as pd
import sklearn
import scipy
import matplotlib
import colorama

is_colab = 'google.colab' in sys.modules
env = 'Colab' if is_colab else 'Local'
print(f'Environment: {env}')
print(f'Python:       {sys.version}')
print()
print(f'numpy:        {np.__version__:<12s} (Colab has 2.0.2)')
print(f'pandas:       {pd.__version__:<12s} (Colab has 2.2.2)')
print(f'scikit-learn: {sklearn.__version__:<12s} (Colab has 1.6.1)')
print(f'scipy:        {scipy.__version__:<12s} (Colab has 1.16.3)')
print(f'matplotlib:   {matplotlib.__version__:<12s} (Colab has 3.10.0)')
print(f'colorama:     {colorama.__version__:<12s} (non-Colab pkg, always installed)')

In [ ]:
# Functional test: verify C extensions actually work (these break when versions are corrupted)
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from scipy import stats, linalg
import pandas as pd

# numpy: dtype operations (breaks with ABI mismatch)
arr = np.array([1.0, 2.0, 3.0], dtype=np.float64)
assert arr.dtype == np.float64
print(f'numpy dtype test:     PASS (float64 array, dtype={arr.dtype})')

# scipy: C extension linalg (breaks with _center import error)
A = np.array([[1, 2], [3, 4]])
inv = linalg.inv(A)
print(f'scipy linalg test:    PASS (matrix inverse computed)')

# sklearn: fit + predict (uses numpy C internals)
X = np.array([[1], [2], [3], [4], [5]])
y = np.array([2.1, 3.9, 6.1, 8.0, 9.9])
model = LinearRegression().fit(X, y)
pred = model.predict([[6]])[0]
print(f'sklearn fit test:     PASS (coef={model.coef_[0]:.2f}, pred[6]={pred:.2f})')

# sklearn: StandardScaler (uses scipy internally)
scaler = StandardScaler().fit(X)
scaled = scaler.transform(X)
print(f'sklearn scaler test:  PASS (mean={scaler.mean_[0]:.1f}, std={scaler.scale_[0]:.2f})')

# scipy: stats (common failure point)
slope, intercept, r, p, se = stats.linregress(X.flatten(), y)
print(f'scipy stats test:     PASS (linregress r\u00b2={r**2:.4f})')

# pandas: DataFrame with numpy integration
df = pd.DataFrame({'x': X.flatten(), 'y': y})
corr = df.corr().iloc[0, 1]
print(f'pandas+numpy test:    PASS (correlation={corr:.4f})')

# matplotlib: import + basic plot (verify no C extension crash)
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(6, 3))
ax.scatter(X.flatten(), y, label='data')
ax.plot([1, 6], [model.predict([[1]])[0], pred], 'r-', label='fit')
ax.set_title('C Extension Smoke Test')
ax.legend()
plt.tight_layout()
plt.show()

print()
print('ALL TESTS PASSED \u2014 C extensions are working correctly!')